In [1]:
# start spark application
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1604423401068_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# install and import pandas lib
sc.install_pypi_package("pandas")
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/bf/4c/cb7da76f3a5e077e545f9cf8575b8f488a4e8ad60490838f89c5cdd5bb57/pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

In [3]:
# load data from S3
df_songs = spark.read.parquet('s3://udacity-de-datalake/songs-table')
df_artists = spark.read.parquet('s3://udacity-de-datalake/artists-table')
df_users = spark.read.parquet('s3://udacity-de-datalake/users-table')
df_time = spark.read.parquet('s3://udacity-de-datalake/time-table')
df_songplays = spark.read.parquet('s3://udacity-de-datalake/songplays_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# create tmp view
df_songs.createOrReplaceTempView('songs_tbl')
df_artists.createOrReplaceTempView('artists_tbl')
df_users.createOrReplaceTempView('users_tbl')
df_time.createOrReplaceTempView('time_tbl')
df_songplays.createOrReplaceTempView('songplays_tbl')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Query 1. Top-10 most played songs

In [5]:
spark.sql('''
  SELECT sp.song_id, s.title, count(*) AS cnt 
    FROM songplays_tbl sp
    JOIN songs_tbl s
      ON sp.song_id = s.song_id
GROUP BY 1, 2
ORDER BY 3 DESC
   LIMIT 10;
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              song_id                                              title  cnt
0  SOBONKR12A58A7A7E0                                     You're The One   37
1  SOUNZHU12A8AE47481                                I CAN'T GET STARTED    9
2  SOHTKMO12AB01843B0  Catch You Baby (Steve Pitron & Max Sanna Radio...    9
3  SOULTKQ12AB018A183  Nothin' On You [feat. Bruno Mars] (Album Version)    8
4  SOLZOBD12AB0185720                           Hey Daddy (Daddy's Home)    6
5  SOTNHIP12AB0183131                                       Make Her Say    5
6  SOARUPP12AB01842E0                                       Up Up & Away    5
7  SOIOESO12A6D4F621D                             Unwell (Album Version)    4
8  SONQEYS12AF72AABC9                                          Mr. Jones    4
9  SOIZLKI12A6D4F7B61            Supermassive Black Hole (Album Version)    4

### Query 2. Top-10 most played artists

In [6]:
spark.sql('''
  SELECT sp.artist_id, a.name AS artist_name, count(*) AS cnt
    FROM songplays_tbl sp
    JOIN artists_tbl a
      ON sp.artist_id = a.artist_id
GROUP BY 1, 2
ORDER BY 3 DESC
   LIMIT 10;
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            artist_id                                     artist_name  cnt
0  AR5E44Z1187B9A1D74                                   Dwight Yoakam   37
1  ARD46C811C8A414F3F                                        Kid Cudi   10
2  ARD46C811C8A414F3F                  Kid Cudi / Kanye West / Common   10
3  AR37SX11187FB3E164                                      Ron Carter    9
4  AR5EYTL1187B98EDA0                                   Lonnie Gordon    9
5  ARKQQZA12086C116FC                                           B.o.B    8
6  ARR3ONV1187B9A2F59                                            Muse    6
7  ARPDVPJ1187B9ADBE9                  Usher featuring Jermaine Dupri    6
8  ARPDVPJ1187B9ADBE9                                           Usher    6
9  ARM0P6Z1187FB4D466  Richard Hawley And Death Ramps_ Arctic Monkeys    5

### Query 3. Statistics on when songs are played during a day

In [7]:
spark.sql('''
  SELECT CASE
           WHEN t.hour BETWEEN 2 AND 8  THEN '2~8'
           WHEN t.hour BETWEEN 9 AND 12 THEN '9~12'
           WHEN t.hour BETWEEN 13 AND 18 THEN '13~18'
           WHEN t.hour BETWEEN 19 AND 22 THEN '19~22'
           ELSE '23~24, 0~2'
         END AS play_time, 
         count(*) AS cnt
    FROM songplays_tbl sp
    JOIN time_tbl t
      ON sp.start_time = t.start_time
GROUP BY 1
ORDER BY 2 DESC;
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    play_time  cnt
0       13~18  145
1         2~8   59
2       19~22   53
3        9~12   48
4  23~24, 0~2   28